In [236]:
from demo  import jdt_db
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import time
from snapshot_selenium import snapshot
from sklearn.ensemble import RandomForestClassifier # RF库
from pyecharts.charts import Bar3D
from pyecharts.globals import CurrentConfig, NotebookType  
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

In [299]:
# 数据导入
data1 = jdt_db("SELECT  user_id,  user_name,  created_at,  money ,user_group       FROM  jdt_user_upgrade_log   WHERE  created_at > '2022-03-01 00:00:00'  AND created_at < '2022-07-01 00:00:00'")
data2 = jdt_db("   SELECT   	a.user_id, 	b.truename,     	FROM_UNIXTIME( a.pay_time, '%Y-%m-%d  %H:%i:%s' ) AS 'time',   	a.pay_price,   	b.group_id    FROM   	jdt_litejdtstore_order AS a   	LEFT JOIN jdt_user AS b ON a.user_id = b.id    WHERE   	FROM_UNIXTIME( a.pay_time, '%Y-%m-%d %H:%i:%s' ) >= '2022-03-01 00:00:00'    	AND FROM_UNIXTIME( a.pay_time, '%Y-%m-%d %H:%i:%s' ) < '2022-07-01 00:00:00'    GROUP BY   	order_no")
data3 = jdt_db("   SELECT   user_id,   count(user_id) as counts   FROM   jdt_user_logon_in   WHERE   created_at  >= '2022-03-01 00:00:00'    AND  created_at < '2022-07-01 00:00:00'       GROUP BY   user_id")

1.1163642406463623


D:\anaconda\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


0.31694936752319336


D:\anaconda\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


11.680164337158203


In [300]:
data2 = data2.rename(columns={'truename':'user_name','time':'created_at','pay_price':'money','group_id':'user_group'})

In [301]:
total_data = pd.concat([data1,data2],axis = 0)

In [302]:
total_data

,user_id,user_name,created_at,money,user_group
0,1716219,张洋,2022-03-01 00:13:24,43199.0,3
1,1889849,陈丽芬,2022-03-01 05:15:02,199.0,1
2,1891100,马兰芳,2022-03-01 07:56:48,199.0,1
3,1888321,姜佩华,2022-03-01 08:18:00,199.0,1
4,556362,朱孝营,2022-03-01 08:39:37,499.0,1
...,...,...,...,...,...
1025,38387,覃小鑫,2022-06-30 22:14:54,5.5,3
1026,38387,覃小鑫,2022-06-30 22:17:25,5.5,3
1027,533334,孙相水,2022-06-30 22:26:26,30.0,5
1028,971179,王怀亮,2022-06-30 22:56:06,5.5,2


In [303]:
total_data =pd.merge(total_data,data3,on='user_id',how = 'left')

In [307]:
total_data

,user_id,user_name,created_at,money,user_group,counts
0,1716219,张洋,2022-03-01 00:13:24,43199.0,3,25.0
1,1889849,陈丽芬,2022-03-01 05:15:02,199.0,1,3.0
2,1891100,马兰芳,2022-03-01 07:56:48,199.0,1,13.0
3,1888321,姜佩华,2022-03-01 08:18:00,199.0,1,3.0
4,556362,朱孝营,2022-03-01 08:39:37,499.0,1,3.0
...,...,...,...,...,...,...
31615,38387,覃小鑫,2022-06-30 22:14:54,5.5,3,5.0
31616,38387,覃小鑫,2022-06-30 22:17:25,5.5,3,5.0
31617,533334,孙相水,2022-06-30 22:26:26,30.0,5,22.0
31618,971179,王怀亮,2022-06-30 22:56:06,5.5,2,5.0


In [308]:
total_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31620 entries, 0 to 31619
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   user_id     31620 non-null  int64  
 1   user_name   31620 non-null  object 
 2   created_at  31620 non-null  object 
 3   money       31620 non-null  float64
 4   user_group  31620 non-null  int64  
 5   counts      31512 non-null  float64
dtypes: float64(2), int64(2), object(2)
memory usage: 1.7+ MB


In [309]:
total_data['created_at'] = pd.to_datetime(total_data['created_at'])

In [310]:
total_data['max_time'] = pd.to_datetime('2022-06-30 23:59:59')

In [311]:
total_data = total_data[total_data['user_group'] < 4]

In [312]:
total_data = total_data[total_data['money'] > 0]

In [313]:
total_data['date_interval'] = total_data['max_time'] - total_data['created_at']

In [314]:
total_data['month'] = total_data['created_at'].dt.month

In [315]:
total_data['date_interval'] = total_data['date_interval'].apply(lambda x: x.days)

In [316]:
total_data

,user_id,user_name,created_at,money,user_group,counts,max_time,date_interval,month
0,1716219,张洋,2022-03-01 00:13:24,43199.0,3,25.0,2022-06-30 23:59:59,121,3
1,1889849,陈丽芬,2022-03-01 05:15:02,199.0,1,3.0,2022-06-30 23:59:59,121,3
2,1891100,马兰芳,2022-03-01 07:56:48,199.0,1,13.0,2022-06-30 23:59:59,121,3
3,1888321,姜佩华,2022-03-01 08:18:00,199.0,1,3.0,2022-06-30 23:59:59,121,3
4,556362,朱孝营,2022-03-01 08:39:37,499.0,1,3.0,2022-06-30 23:59:59,121,3
...,...,...,...,...,...,...,...,...,...
31612,592013,龙小年,2022-06-30 21:39:43,30.0,2,6.0,2022-06-30 23:59:59,0,6
31614,43931,孙学娟,2022-06-30 21:46:16,5.5,3,10.0,2022-06-30 23:59:59,0,6
31615,38387,覃小鑫,2022-06-30 22:14:54,5.5,3,5.0,2022-06-30 23:59:59,0,6
31616,38387,覃小鑫,2022-06-30 22:17:25,5.5,3,5.0,2022-06-30 23:59:59,0,6


In [317]:
rfm_gb = total_data.groupby(['month','user_id'],as_index=False).agg(
    {'date_interval': 'min',  # 计算最近一次订单时间
     'counts': 'max', # 计算订单频率
     'money': 'sum',
    'user_group':'max'})  # 计算订单总金额

In [318]:
rfm_gb_group = rfm_gb[rfm_gb['user_group']<4]

In [319]:
rfm_gb_group

,month,user_id,date_interval,counts,money,user_group
0,3,14141,121,13.0,33250.0,3
1,3,15793,121,6.0,30.0,3
2,3,18593,121,8.0,43199.0,3
3,3,25749,113,2.0,30.0,3
4,3,26601,104,11.0,5198.0,2
...,...,...,...,...,...,...
25831,6,2140706,0,2.0,199.0,1
25832,6,2140742,0,5.0,499.0,1
25833,6,2140746,0,3.0,499.0,1
25834,6,2140776,0,3.0,5498.0,2


In [320]:
rfm_gb.columns =  ['month','user_id','r','f','m','user_group']
rfm_gb

,month,user_id,r,f,m,user_group
0,3,14141,121,13.0,33250.0,3
1,3,15793,121,6.0,30.0,3
2,3,18593,121,8.0,43199.0,3
3,3,25749,113,2.0,30.0,3
4,3,26601,104,11.0,5198.0,2
...,...,...,...,...,...,...
25831,6,2140706,0,2.0,199.0,1
25832,6,2140742,0,5.0,499.0,1
25833,6,2140746,0,3.0,499.0,1
25834,6,2140776,0,3.0,5498.0,2


In [321]:
desc_pd = rfm_gb.iloc[:,2:].describe().T
print(desc_pd)

              count         mean           std  min    25%    50%     75%  \
r           25836.0    55.621652     33.204569  0.0   26.0   54.0    77.0   
f           25734.0     8.549079      9.896840  1.0    3.0    6.0    10.0   
m           25836.0  4650.256566  10899.751321  5.5  199.0  499.0  4999.0   
user_group  25836.0     1.412796      0.627125  1.0    1.0    1.0     2.0   

                max  
r             121.0  
f             237.0  
m           48697.0  
user_group      3.0  


In [328]:
r_bins = [-1,26,77,121] # 注意起始边界小于最小值
f_bins = [0,3,10,237] 
m_bins = [5,199,4999,48697]

In [329]:
# rfm_merge = pd.merge(rfm_gb,total_data,on='user_id',how='inner')

In [334]:
rfm_merge = rfm_gb

In [339]:
rfm_merge.isnull().any()

month         False
user_id       False
r             False
f              True
m             False
user_group    False
dtype: bool

In [340]:
rfm_merge['f'] = rfm_merge['f'].fillna(1)

In [341]:
# 计算权重因子
clf = RandomForestClassifier()
clf = clf.fit(rfm_merge[['r','f','m']],rfm_merge['user_group'])
weights = clf.feature_importances_
print('feature importance:',weights)

feature importance: [0.05426309 0.08279247 0.86294444]


In [342]:
rfm_gb

,month,user_id,r,f,m,user_group
0,3,14141,121,13.0,33250.0,3
1,3,15793,121,6.0,30.0,3
2,3,18593,121,8.0,43199.0,3
3,3,25749,113,2.0,30.0,3
4,3,26601,104,11.0,5198.0,2
...,...,...,...,...,...,...
25831,6,2140706,0,2.0,199.0,1
25832,6,2140742,0,5.0,499.0,1
25833,6,2140746,0,3.0,499.0,1
25834,6,2140776,0,3.0,5498.0,2


In [343]:
# 计算分箱得分
rfm_gb['r_score'] = pd.cut(rfm_gb['r'], r_bins, labels=[i for i in range(len(r_bins)-1,0,-1)])  

rfm_gb['m_score'] = pd.cut(rfm_gb['m'], m_bins, labels=[i+1 for i in range(len(m_bins)-1)])  

rfm_gb['f_score'] = pd.cut(rfm_gb['f'], f_bins, labels=[i+1 for i in range(len(f_bins)-1)]) 

In [344]:
rfm_gb

,month,user_id,r,f,m,user_group,r_score,m_score,f_score
0,3,14141,121,13.0,33250.0,3,1,3,3
1,3,15793,121,6.0,30.0,3,1,1,2
2,3,18593,121,8.0,43199.0,3,1,3,2
3,3,25749,113,2.0,30.0,3,1,1,1
4,3,26601,104,11.0,5198.0,2,1,3,3
...,...,...,...,...,...,...,...,...,...
25831,6,2140706,0,2.0,199.0,1,3,1,1
25832,6,2140742,0,5.0,499.0,1,3,2,2
25833,6,2140746,0,3.0,499.0,1,3,2,1
25834,6,2140776,0,3.0,5498.0,2,3,3,1


In [345]:
rfm_gb = rfm_gb.apply(np.int32) 
rfm_gb['rfm_score'] = rfm_gb['r_score'] * weights[0] + rfm_gb['f_score'] * weights[1] + rfm_gb['m_score'] * weights[2]

In [346]:
rfm_gb

,month,user_id,r,f,m,user_group,r_score,m_score,f_score,rfm_score
0,3,14141,121,13,33250,3,1,3,3,2.891474
1,3,15793,121,6,30,3,1,1,2,1.082792
2,3,18593,121,8,43199,3,1,3,2,2.808681
3,3,25749,113,2,30,3,1,1,1,1.000000
4,3,26601,104,11,5198,2,1,3,3,2.891474
...,...,...,...,...,...,...,...,...,...,...
25831,6,2140706,0,2,199,1,3,1,1,1.108526
25832,6,2140742,0,5,499,1,3,2,2,2.054263
25833,6,2140746,0,3,499,1,3,2,1,1.971471
25834,6,2140776,0,3,5498,2,3,3,1,2.834415


In [347]:
rfm_gb['r_score'] = rfm_gb['r_score'].astype(np.str)
rfm_gb['f_score'] = rfm_gb['f_score'].astype(np.str)
rfm_gb['m_score'] = rfm_gb['m_score'].astype(np.str)
rfm_gb['rfm_group'] = rfm_gb['r_score'].str.cat(rfm_gb['f_score']).str.cat(
rfm_gb['m_score'])

C:\Users\cy176\AppData\Local\Temp\ipykernel_12016\348090419.py:1: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  rfm_gb['r_score'] = rfm_gb['r_score'].astype(np.str)
C:\Users\cy176\AppData\Local\Temp\ipykernel_12016\348090419.py:2: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  rfm_gb['f_score'] = rfm_gb['f_score'].astype(np.str)
C:\Users\cy176\AppData\Local

In [348]:
total_data = total_data[['user_id','user_name']]

In [349]:
total_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30060 entries, 0 to 31618
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    30060 non-null  int64 
 1   user_name  30060 non-null  object
dtypes: int64(1), object(1)
memory usage: 704.5+ KB


In [350]:
rfm_gb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25836 entries, 0 to 25835
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   month       25836 non-null  int32  
 1   user_id     25836 non-null  int32  
 2   r           25836 non-null  int32  
 3   f           25836 non-null  int32  
 4   m           25836 non-null  int32  
 5   user_group  25836 non-null  int32  
 6   r_score     25836 non-null  object 
 7   m_score     25836 non-null  object 
 8   f_score     25836 non-null  object 
 9   rfm_score   25836 non-null  float64
 10  rfm_group   25836 non-null  object 
dtypes: float64(1), int32(6), object(4)
memory usage: 1.6+ MB


In [351]:
rfm_gb_ = pd.merge(rfm_gb,total_data,on='user_id',how = 'left')
rfm_gb_ = rfm_gb_.drop_duplicates(subset='user_id',inplace=False)

In [352]:
rfm_gb_ = rfm_gb_[['user_id','user_name','user_group','r_score','f_score','m_score','rfm_group','rfm_score']]
rfm_gb_

,user_id,user_name,user_group,r_score,f_score,m_score,rfm_group,rfm_score
0,14141,黄群英,3,1,3,3,133,2.891474
1,15793,刘改平,3,1,2,1,121,1.082792
2,18593,吴亦好,3,1,2,3,123,2.808681
3,25749,师湘栋,3,1,1,1,111,1.000000
4,26601,唐爱云,2,1,3,3,133,2.891474
...,...,...,...,...,...,...,...,...
34677,2140706,胡西玲,1,3,1,1,311,1.108526
34678,2140742,黄慧玲,1,3,2,2,322,2.054263
34679,2140746,13652501817,1,3,1,2,312,1.971471
34680,2140776,袁悟清,2,3,1,3,313,2.834415


In [353]:
display_data = rfm_gb.groupby(['rfm_group','month'],as_index=False)['user_id'].count()
display_data.columns = ['rfm_group','month','number']
display_data['rfm_group'] = display_data['rfm_group'].astype(np.int32)
display_data.head()

,rfm_group,month,number
0,111,3,370
1,111,4,173
2,112,3,524
3,112,4,236
4,113,3,32


In [354]:
rfm_user_id_list = list(rfm_gb['user_id'])

In [355]:
rfm_user_id_str = ' ,'.join('%s' %id for id in rfm_user_id_list)

In [356]:
display_data

,rfm_group,month,number
0,111,3,370
1,111,4,173
2,112,3,524
3,112,4,236
4,113,3,32
5,113,4,21
6,121,3,847
7,121,4,281
8,122,3,882
9,122,4,300


In [357]:
display_data.to_excel(r'D:\Work\data\2022.06.17六月份数据分析报告\RFM个级别用户总数.xlsx')
rfm_gb_.to_excel(r'D:\Work\data\2022.06.17六月份数据分析报告\RFM个级别用户详情.xlsx')

In [360]:
import random
import pyecharts.options as opts
from pyecharts.charts import Bar3D

data = display_data.values.tolist() 


c=(
    Bar3D(init_opts=opts.InitOpts(width="900px", height="600px"))
    .add(
        series_name="",
        data=data,
        xaxis3d_opts=opts.Axis3DOpts(type_="category",data = display_data['rfm_group']),
        yaxis3d_opts=opts.Axis3DOpts(type_="category", data = display_data['month']),
        zaxis3d_opts=opts.Axis3DOpts(type_="value"),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts("健康帮RFM价值分层柱状3D图"),
        visualmap_opts=opts.VisualMapOpts(
            max_=2000,
            range_color=[
                "#313695",
                "#4575b4",
                "#74add1",
                "#abd9e9",
                "#e0f3f8",
                "#ffffbf",
                "#fee090",
                "#fdae61",
                "#f46d43",
                "#d73027",
                "#a50026",
            ],
        )
    )
)
c.load_javascript()



In [292]:
make_snapshot(snapshot,c.render(),'1.png')

NameError: name 'make_snapshot' is not defined

In [297]:
c.render_notebook()

In [361]:
c.render_notebook()